In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import nltk
import re

In [2]:
data_full = pd.read_csv('train_data.csv')

In [3]:
data_full.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [4]:
data_full.shape

(7613, 5)

test tweettokenizer

In [12]:
from nltk.tokenize import TweetTokenizer

In [13]:
tt = TweetTokenizer()

In [14]:
data['tweet_tokenized_text'] = data['text'].apply(lambda sent: tt.tokenize(sent))

C:\Users\yunes\AppData\Local\Temp\ipykernel_18416\781003934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet_tokenized_text'] = data['text'].apply(lambda sent: tt.tokenize(sent))


filtering out stop words, punctuation

In [15]:
from nltk.corpus import stopwords
from string import punctuation

In [16]:
stopwords_set = set(stopwords.words("english"))

In [17]:
punctuation_set = set(punctuation)

In [18]:
noise = stopwords_set.union(punctuation_set)

In [19]:
data['filtered_tt_text'] = data['tweet_tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens if token.lower() not in noise])

C:\Users\yunes\AppData\Local\Temp\ipykernel_18416\1276454103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['filtered_tt_text'] = data['tweet_tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens if token.lower() not in noise])


In [20]:
data

,text,target,tweet_tokenized_text,filtered_tt_text
0,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #eart...","[deeds, reason, #earthquake, may, allah, forgi..."
1,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[forest, fire, near, la, ronge, sask, canada]"
2,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, ', shelter, in, pl...","[residents, asked, shelter, place, notified, o..."
3,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #wildfires, evacuati...","[13,000, people, receive, #wildfires, evacuati..."
4,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #Al...","[got, sent, photo, ruby, #alaska, smoke, #wild..."
...,...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,1,"[Two, giant, cranes, holding, a, bridge, colla...","[two, giant, cranes, holding, bridge, collapse..."
7609,@aria_ahrary @TheTawniest The out of control w...,1,"[@aria_ahrary, @TheTawniest, The, out, of, con...","[@aria_ahrary, @thetawniest, control, wild, fi..."
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[M1, ., 94, [, 01:04, UTC, ], ?, 5km, S, of, V...","[m1, 94, 01:04, utc, 5km, volcano, hawaii, htt..."
7611,Police investigating after an e-bike collided ...,1,"[Police, investigating, after, an, e-bike, col...","[police, investigating, e-bike, collided, car,..."


In [21]:
data['filtered_tt_text_joined'] = data['filtered_tt_text'].apply(lambda tokens: " ".join(tokens))

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train_tt, X_val_tt, y_train_tt, y_val_tt = train_test_split(data['filtered_tt_text_joined'], data['target'], test_size=0.2, random_state=42)

In [24]:
X_train_tt.shape, y_train_tt.shape, X_val_tt.shape, y_val_tt.shape

((6090,), (6090,), (1523,), (1523,))

TfIdfVectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
tfidf_sklearn = TfidfVectorizer()

In [27]:
X_train_tt_tfidf = tfidf_sklearn.fit_transform(X_train_tt)

In [28]:
X_val_tt_tfidf = tfidf_sklearn.transform(X_val_tt)

In [29]:
print(X_train_tt_tfidf.shape)
print(X_val_tt_tfidf.shape)

(6090, 18452)
(1523, 18452)


Bag of words vectorizer

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
count_vectorizer = CountVectorizer()

In [32]:
X_train_tt_bow = count_vectorizer.fit_transform(X_train_tt)

In [33]:
X_val_tt_bow = count_vectorizer.transform(X_val_tt)

In [34]:
print(X_train_tt_bow.shape)
print(X_val_tt_bow.shape)

(6090, 18452)
(1523, 18452)


ML model

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
lr_tt = LogisticRegression(max_iter=1000, random_state=42)

In [38]:
param_grid = {
    'C': [0.1, 1, 10, 100]
}

In [39]:
grid_search_tt = GridSearchCV(lr_tt, param_grid, cv=5, scoring='f1', n_jobs=-1)

In [40]:
grid_search_tt.fit(X_train_tt_tfidf, y_train_tt)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1, param_grid={'C': [0.1, 1, 10, 100]}, scoring='f1')

In [41]:
grid_search_tt.best_params_

{'C': 10}

In [42]:
grid_search_tt.best_score_

0.7361439373391462

In [43]:
y_pred_tt_val = grid_search_tt.predict(X_val_tt_tfidf)

In [44]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [45]:
print(classification_report(y_val_tt, y_pred_tt_val))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       874
           1       0.75      0.74      0.74       649

    accuracy                           0.78      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.78      0.78      0.78      1523



testing different tokenizer and preprocessing steps

In [46]:
def test_preproc(data, tokenizer, note=''):
    print(note)
    data = data.copy()
    
    data['tokenized_text'] = data['text'].apply(
        lambda sent: tokenizer.tokenize(sent) if hasattr(tokenizer, 'tokenize') else tokenizer(sent)
    )
    
    stopwords_set = set(stopwords.words("english"))
    punctuation_set = set(punctuation)
    noise = stopwords_set.union(punctuation_set)
    
    data['filtered_text'] = data['tokenized_text'].apply(
        lambda tokens: [token.lower() for token in tokens if token.lower() not in noise]
    )
    
    data['filtered_text_joined'] = data['filtered_text'].apply(lambda tokens: " ".join(tokens))
    
    X_train, X_val, y_train, y_val = train_test_split(
        data['filtered_text_joined'], data['target'], test_size=0.2, random_state=42
    )
    
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_val_tfidf = tfidf_vectorizer.transform(X_val)
    
    count_vectorizer = CountVectorizer()
    X_train_bow = count_vectorizer.fit_transform(X_train)
    X_val_bow = count_vectorizer.transform(X_val)
    
    lr_tfidf = LogisticRegression(max_iter=1000, random_state=42)
    param_grid = {'C': [0.1, 1, 10, 100]}
    
    grid_search_tfidf = GridSearchCV(lr_tfidf, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search_tfidf.fit(X_train_tfidf, y_train)
    
    print("best cross-val f1 score with TfIdf vectorizer:", grid_search_tfidf.best_score_)
    
    y_pred_tfidf = grid_search_tfidf.predict(X_val_tfidf)
    print("val acc with TfIdf:", accuracy_score(y_val, y_pred_tfidf))
    print("val f1-score with TfIdf:", f1_score(y_val, y_pred_tfidf))
    print("classification report with TfIdf:\n", classification_report(y_val, y_pred_tfidf))
    print("---")


    lr_bow = LogisticRegression(max_iter=1000, random_state=42)
    param_grid = {'C': [0.1, 1, 10, 100]}
    
    grid_search_bow = GridSearchCV(lr_bow, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search_bow.fit(X_train_bow, y_train)

    print("best cross-val f1 score with BoW vectorizer:", grid_search_bow.best_score_)
    
    y_pred_bow = grid_search_tfidf.predict(X_val_bow)
    print("val acc with BoW:", accuracy_score(y_val, y_pred_bow))
    print("val f1-score with BoW:", f1_score(y_val, y_pred_bow))
    print("classification report with BoW:\n", classification_report(y_val, y_pred_bow))

In [47]:
test_preproc(data, tt, note='Classification using TweetTokenizer')

Classification using TweetTokenizer
best cross-val f1 score with TfIdf vectorizer: 0.7361439373391462
val acc with TfIdf: 0.7839789888378201
val f1-score with TfIdf: 0.7447633824670287
classification report with TfIdf:
               precision    recall  f1-score   support

           0       0.81      0.82      0.81       874
           1       0.75      0.74      0.74       649

    accuracy                           0.78      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.78      0.78      0.78      1523

---
best cross-val f1 score with BoW vectorizer: 0.7355854495829642
val acc with BoW: 0.6631648063033486
val f1-score with BoW: 0.6926303175554224
classification report with BoW:
               precision    recall  f1-score   support

           0       0.86      0.49      0.63       874
           1       0.57      0.89      0.69       649

    accuracy                           0.66      1523
   macro avg       0.71      0.69      0.66      1523
w

In [48]:
from nltk.stem.snowball import EnglishStemmer

In [49]:
tknzr = TweetTokenizer()
stemmer = EnglishStemmer()

In [50]:
def tokenize_and_stemm(text):
    tokens = tknzr.tokenize(text)
    return list(map(stemmer.stem, tokens))

In [51]:
test_preproc(data, tokenize_and_stemm, 'Classification with TweetTokenizer and stemming')


Classification with TweetTokenizer and stemming
best cross-val f1 score with TfIdf vectorizer: 0.7380054778925823
val acc with TfIdf: 0.7728168089297439
val f1-score with TfIdf: 0.7317829457364341
classification report with TfIdf:
               precision    recall  f1-score   support

           0       0.80      0.81      0.80       874
           1       0.74      0.73      0.73       649

    accuracy                           0.77      1523
   macro avg       0.77      0.77      0.77      1523
weighted avg       0.77      0.77      0.77      1523

---
best cross-val f1 score with BoW vectorizer: 0.7438391571563088
val acc with BoW: 0.6611950098489823
val f1-score with BoW: 0.6928571428571428
classification report with BoW:
               precision    recall  f1-score   support

           0       0.86      0.49      0.62       874
           1       0.56      0.90      0.69       649

    accuracy                           0.66      1523
   macro avg       0.71      0.69      0.66

In [52]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yunes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [53]:
tknzr = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

In [54]:
def tokenize_and_lemmatize(text):
    tokens = tknzr.tokenize(text)
    return list(map(lemmatizer.lemmatize, tokens))

In [55]:
test_preproc(data, tokenize_and_lemmatize,  'Classification with TweetTokenizer and lemmatization')

Classification with TweetTokenizer and lemmatization
best cross-val f1 score with TfIdf vectorizer: 0.7343460162012873
val acc with TfIdf: 0.7859487852921865
val f1-score with TfIdf: 0.7461059190031153
classification report with TfIdf:
               precision    recall  f1-score   support

           0       0.81      0.82      0.81       874
           1       0.75      0.74      0.75       649

    accuracy                           0.79      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523

---
best cross-val f1 score with BoW vectorizer: 0.735748389432159
val acc with BoW: 0.6625082074852265
val f1-score with BoW: 0.6944114149821641
classification report with BoW:
               precision    recall  f1-score   support

           0       0.87      0.49      0.62       874
           1       0.57      0.90      0.69       649

    accuracy                           0.66      1523
   macro avg       0.72      0.69      

In [56]:
from nltk.tokenize import word_tokenize

tknzr = word_tokenize
lemmatizer = WordNetLemmatizer()

In [57]:
def tokenize_and_lemmatize(text):
    tokens = tknzr(text)
    return list(map(lemmatizer.lemmatize, tokens))

In [58]:
test_preproc(data, tokenize_and_lemmatize,  'Classification with word_tokenizer and lemmatization')

Classification with word_tokenizer and lemmatization
best cross-val f1 score with TfIdf vectorizer: 0.7346059136185525
val acc with TfIdf: 0.7866053841103086
val f1-score with TfIdf: 0.7446975648075412
classification report with TfIdf:
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       874
           1       0.76      0.73      0.74       649

    accuracy                           0.79      1523
   macro avg       0.78      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523

---
best cross-val f1 score with BoW vectorizer: 0.7359799999400071
val acc with BoW: 0.6677609980302035
val f1-score with BoW: 0.6977299880525687
classification report with BoW:
               precision    recall  f1-score   support

           0       0.87      0.50      0.63       874
           1       0.57      0.90      0.70       649

    accuracy                           0.67      1523
   macro avg       0.72      0.70     

*Conclusion*: 

the best quality on validation set is achieved using TweetTokenizer and lemmatization as preprocessing steps and using TfIdf vectorizer (f1_val = 0.7447)

Getting predictions on test sample with the best model

In [59]:
tknzr = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

In [60]:
def tokenize_and_lemmatize(text):
    tokens = tknzr.tokenize(text)
    return list(map(lemmatizer.lemmatize, tokens))

In [61]:
tokenizer = tokenize_and_lemmatize

In [62]:
data = data.copy()
    
data['tokenized_text'] = data['text'].apply(
    lambda sent: tokenizer(sent)
)

stopwords_set = set(stopwords.words("english"))
punctuation_set = set(punctuation)
noise = stopwords_set.union(punctuation_set)

data['filtered_text'] = data['tokenized_text'].apply(
    lambda tokens: [token.lower() for token in tokens if token.lower() not in noise]
)

data['filtered_text_joined'] = data['filtered_text'].apply(lambda tokens: " ".join(tokens))

X_train, X_val, y_train, y_val = train_test_split(
    data['filtered_text_joined'], data['target'], test_size=0.2, random_state=42
)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

lr_tfidf = LogisticRegression(max_iter=1000, random_state=42)
param_grid = {'C': [0.1, 1, 10, 100]}

grid_search_tfidf = GridSearchCV(lr_tfidf, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search_tfidf.fit(X_train_tfidf, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1, param_grid={'C': [0.1, 1, 10, 100]}, scoring='f1')

In [63]:
data

,text,target,tweet_tokenized_text,filtered_tt_text,filtered_tt_text_joined,tokenized_text,filtered_text,filtered_text_joined
0,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #eart...","[deeds, reason, #earthquake, may, allah, forgi...",deeds reason #earthquake may allah forgive us,"[Our, Deeds, are, the, Reason, of, this, #eart...","[deeds, reason, #earthquake, may, allah, forgi...",deeds reason #earthquake may allah forgive u
1,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada,"[Forest, fire, near, La, Ronge, Sask, ., Canada]","[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, ', shelter, in, pl...","[residents, asked, shelter, place, notified, o...",residents asked shelter place notified officer...,"[All, resident, asked, to, ', shelter, in, pla...","[resident, asked, shelter, place, notified, of...",resident asked shelter place notified officer ...
3,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #wildfires, evacuati...","[13,000, people, receive, #wildfires, evacuati...","13,000 people receive #wildfires evacuation or...","[13,000, people, receive, #wildfires, evacuati...","[13,000, people, receive, #wildfires, evacuati...","13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #Al...","[got, sent, photo, ruby, #alaska, smoke, #wild...",got sent photo ruby #alaska smoke #wildfires p...,"[Just, got, sent, this, photo, from, Ruby, #Al...","[got, sent, photo, ruby, #alaska, smoke, #wild...",got sent photo ruby #alaska smoke #wildfires p...
...,...,...,...,...,...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,1,"[Two, giant, cranes, holding, a, bridge, colla...","[two, giant, cranes, holding, bridge, collapse...",two giant cranes holding bridge collapse nearb...,"[Two, giant, crane, holding, a, bridge, collap...","[two, giant, crane, holding, bridge, collapse,...",two giant crane holding bridge collapse nearby...
7609,@aria_ahrary @TheTawniest The out of control w...,1,"[@aria_ahrary, @TheTawniest, The, out, of, con...","[@aria_ahrary, @thetawniest, control, wild, fi...",@aria_ahrary @thetawniest control wild fires c...,"[@aria_ahrary, @TheTawniest, The, out, of, con...","[@aria_ahrary, @thetawniest, control, wild, fi...",@aria_ahrary @thetawniest control wild fire ca...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[M1, ., 94, [, 01:04, UTC, ], ?, 5km, S, of, V...","[m1, 94, 01:04, utc, 5km, volcano, hawaii, htt...",m1 94 01:04 utc 5km volcano hawaii http://t.co...,"[M1, ., 94, [, 01:04, UTC, ], ?, 5km, S, of, V...","[m1, 94, 01:04, utc, 5km, volcano, hawaii, htt...",m1 94 01:04 utc 5km volcano hawaii http://t.co...
7611,Police investigating after an e-bike collided ...,1,"[Police, investigating, after, an, e-bike, col...","[police, investigating, e-bike, collided, car,...",police investigating e-bike collided car littl...,"[Police, investigating, after, an, e-bike, col...","[police, investigating, e-bike, collided, car,...",police investigating e-bike collided car littl...


In [64]:
test_data = pd.read_csv('test_data.csv')[['text']]

In [65]:
test_data['tokenized_text'] = test_data['text'].apply(
    lambda sent: tokenizer(sent)
)

In [66]:
test_data['filtered_text'] = test_data['tokenized_text'].apply(
    lambda tokens: [token.lower() for token in tokens if token.lower() not in noise]
)

test_data['filtered_text_joined'] = test_data['filtered_text'].apply(lambda tokens: " ".join(tokens))

In [67]:
test_data

,text,tokenized_text,filtered_text,filtered_text_joined
0,Just happened a terrible car crash,"[Just, happened, a, terrible, car, crash]","[happened, terrible, car, crash]",happened terrible car crash
1,"Heard about #earthquake is different cities, s...","[Heard, about, #earthquake, is, different, cit...","[heard, #earthquake, different, city, stay, sa...",heard #earthquake different city stay safe eve...
2,"there is a forest fire at spot pond, geese are...","[there, is, a, forest, fire, at, spot, pond, ,...","[forest, fire, spot, pond, goose, fleeing, acr...",forest fire spot pond goose fleeing across str...
3,Apocalypse lighting. #Spokane #wildfires,"[Apocalypse, lighting, ., #Spokane, #wildfires]","[apocalypse, lighting, #spokane, #wildfires]",apocalypse lighting #spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan,"[Typhoon, Soudelor, kill, 28, in, China, and, ...","[typhoon, soudelor, kill, 28, china, taiwan]",typhoon soudelor kill 28 china taiwan
...,...,...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,"[EARTHQUAKE, SAFETY, LOS, ANGELES, , ÛÒ, SAFE...","[earthquake, safety, los, angeles, , ûò, safe...",earthquake safety los angeles  ûò safety fast...
3259,Storm in RI worse than last hurricane. My city...,"[Storm, in, RI, worse, than, last, hurricane, ...","[storm, ri, worse, last, hurricane, city, 3oth...",storm ri worse last hurricane city 3others har...
3260,Green Line derailment in Chicago http://t.co/U...,"[Green, Line, derailment, in, Chicago, http://...","[green, line, derailment, chicago, http://t.co...",green line derailment chicago http://t.co/utbx...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,"[MEG, issue, Hazardous, Weather, Outlook, (, H...","[meg, issue, hazardous, weather, outlook, hwo,...",meg issue hazardous weather outlook hwo http:/...


In [68]:
X_test_tfidf = tfidf_vectorizer.transform(test_data['filtered_text_joined'])

In [69]:
y_test_pred_tfidf = grid_search_tfidf.predict(X_test_tfidf)

In [70]:
sample_submission = pd.read_csv('sample_submission.csv')

In [71]:
test_submission_tdidf = pd.DataFrame(y_test_pred_tfidf, index=sample_submission.id, columns=['target'])

In [72]:
test_submission_tdidf.index.name = 'id'

In [73]:
test_submission_tdidf.to_csv('test_submission_tdidf.csv')

| preprocessing              | vectorizer | best CV f1 score | validation acc | validation f1 score |
| ------------------------------ | ---------- | ---------- | ------------------- | ------------- |
| TweetTokenizer                 | TfIdf      | 0.73614    | 0.78398             | 0.74476       |
| TweetTokenizer                 | BoW        | 0.73559    | 0.66316             | 0.69263       |
| TweetTokenizer + stemming      | TfIdf      | 0.73801    | 0.77282             | 0.73178       |
| TweetTokenizer + stemming      | BoW        | 0.74384    | 0.66120             | 0.69286       |
| TweetTokenizer + lemmatization | TfIdf      | 0.73435    | 0.78595             | 0.74611       |
| TweetTokenizer + lemmatization | BoW        | 0.73575    | 0.66251             | 0.69441       |
| word_tokenize + lemmatization  | TfIdf      | 0.73461    | 0.78661             | 0.74470       |
| word_tokenize + lemmatization  | BoW        | 0.73598    | 0.66776             | 0.69773       |


public score on kaggle test dataset: 0.78945